In [1]:
import cv2
import numpy as np
import os
import random
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img


In [2]:

def augment_angleWave_image(image, output_folder, base_name="angle_1", augment_count=2):
    h, w = image.shape[:2]
    os.makedirs(output_folder, exist_ok=True)

    # Get the current max index to continue from
    existing_files = [f for f in os.listdir(output_folder) if f.startswith(base_name) and f.endswith(".jpg")]
    existing_indices = [
        int(f.split("_")[-1].split(".")[0])
        for f in existing_files if f.split("_")[-1].split(".")[0].isdigit()
    ]
    start_index = max(existing_indices, default=-1) + 1

    for i in range(augment_count):
        img_aug = image.copy()

        # Random shift
        dx = random.randint(-10, 10)
        dy = random.randint(-10, 10)
        matrix = np.float32([[1, 0, dx], [0, 1, dy]])
        img_aug = cv2.warpAffine(img_aug, matrix, (w, h), borderMode=cv2.BORDER_REFLECT)

        # Slight random shift
        dx = random.randint(-5, 5)
        dy = random.randint(-5, 5)
        matrix = np.float32([[1, 0, dx], [0, 1, dy]])
        img_aug = cv2.warpAffine(img_aug, matrix, (w, h), borderMode=cv2.BORDER_REFLECT)

        # Slight random zoom
        zoom_factor = random.uniform(0.95, 1.05)
        zoomed = cv2.resize(img_aug, None, fx=zoom_factor, fy=zoom_factor)
        zh, zw = zoomed.shape[:2]
        if zoom_factor > 1:
            # Crop center
            startx = (zw - w) // 2
            starty = (zh - h) // 2
            img_aug = zoomed[starty:starty+h, startx:startx+w]
        else:
            # Pad
            padx = (w - zw) // 2
            pady = (h - zh) // 2
            img_aug = cv2.copyMakeBorder(zoomed, pady, h - zh - pady, padx, w - zw - padx, cv2.BORDER_REFLECT)

        # Slight random brightness change
        brightness = random.uniform(0.8, 1.2)
        img_aug = np.clip(img_aug * brightness, 0, 255).astype(np.uint8)

        # Slight random rotation
        angle = random.uniform(-5, 5)
        rot_matrix = cv2.getRotationMatrix2D((w // 2, h // 2), angle, 1.0)
        img_aug = cv2.warpAffine(img_aug, rot_matrix, (w, h), borderMode=cv2.BORDER_REFLECT)
        # Save with incremented name
        filename = f"{base_name}_{start_index + i:04d}.jpg"
        cv2.imwrite(os.path.join(output_folder, filename), img_aug)

    print(f"[INFO] Saved {augment_count} images starting from index {start_index} in {output_folder}")


In [8]:
#"C:\Users\josep\OneDrive\Desktop\angles\angle_1.jpg"
#f"C:/Users/josep/OneDrive/Pictures/Camera Roll/angle_{i}P.jpg"
# === Loop through angles 1 to 7 ===
for i in range(1, 7):  # from 1 to 6 inclusive
    img_path =f"C:/Users/josep/OneDrive/Desktop/angles/angle_{i}I.jpg"
    img = cv2.imread(img_path)

    if img is None:
        print(f"⚠️ Could not read image: {img_path}")
        continue

    output_folder = f"dataset1/angle_{i}"
    base_name = f"angle_{i}"

    augment_angleWave_image(
        image=img,
        output_folder=output_folder,
        base_name=base_name,
        augment_count=50
    )

    print(f"✅ Augmented angle_{i} and saved to {output_folder}")


[INFO] Saved 50 images starting from index 250 in dataset1/angle_1
✅ Augmented angle_1 and saved to dataset1/angle_1
[INFO] Saved 50 images starting from index 250 in dataset1/angle_2
✅ Augmented angle_2 and saved to dataset1/angle_2
[INFO] Saved 50 images starting from index 250 in dataset1/angle_3
✅ Augmented angle_3 and saved to dataset1/angle_3
[INFO] Saved 50 images starting from index 250 in dataset1/angle_4
✅ Augmented angle_4 and saved to dataset1/angle_4
[INFO] Saved 50 images starting from index 250 in dataset1/angle_5
✅ Augmented angle_5 and saved to dataset1/angle_5
[INFO] Saved 50 images starting from index 250 in dataset1/angle_6
✅ Augmented angle_6 and saved to dataset1/angle_6


In [9]:
import cv2
import numpy as np
import os
import random

def augment_noWave_image(image, output_folder, base_name="noWave", augment_count=2):
    h, w = image.shape[:2]
    os.makedirs(output_folder, exist_ok=True)

    # Get the current max index to continue from
    existing_files = [f for f in os.listdir(output_folder) if f.startswith(base_name) and f.endswith(".jpg")]
    existing_indices = [
        int(f.split("_")[-1].split(".")[0])
        for f in existing_files if f.split("_")[-1].split(".")[0].isdigit()
    ]
    start_index = max(existing_indices, default=-1) + 1

    for i in range(augment_count):
        img_aug = image.copy()

        # Random horizontal flip
        if random.random() > 0.5:
            img_aug = cv2.flip(img_aug, 1)

        # Random shift
        dx = random.randint(-10, 10)
        dy = random.randint(-10, 10)
        matrix = np.float32([[1, 0, dx], [0, 1, dy]])
        img_aug = cv2.warpAffine(img_aug, matrix, (w, h), borderMode=cv2.BORDER_REFLECT)

        # Random zoom
        zoom_factor = random.uniform(0.9, 1.1)
        zoomed = cv2.resize(img_aug, None, fx=zoom_factor, fy=zoom_factor)
        zh, zw = zoomed.shape[:2]
        if zoom_factor > 1:
            startx = (zw - w) // 2
            starty = (zh - h) // 2
            img_aug = zoomed[starty:starty+h, startx:startx+w]
        else:
            padx = (w - zw) // 2
            pady = (h - zh) // 2
            img_aug = cv2.copyMakeBorder(zoomed, pady, h - zh - pady, padx, w - zw - padx, cv2.BORDER_REFLECT)

        # Random brightness & contrast
        brightness = random.uniform(0.8, 1.2)
        contrast = random.uniform(0.8, 1.2)
        img_aug = np.clip((img_aug - 127.5) * contrast + 127.5 * brightness, 0, 255).astype(np.uint8)

        # Random small rotation
        angle = random.uniform(-10, 10)
        rot_matrix = cv2.getRotationMatrix2D((w // 2, h // 2), angle, 1.0)
        img_aug = cv2.warpAffine(img_aug, rot_matrix, (w, h), borderMode=cv2.BORDER_REFLECT)

        # Optional: Random blur or noise
        #if random.random() < 0.3:
        #    kernel_size = random.choice([3, 5])
        #    img_aug = cv2.GaussianBlur(img_aug, (kernel_size, kernel_size), 0)
        #if random.random() < 0.2:
        #    noise = np.random.normal(0, 10, img_aug.shape).astype(np.uint8)
        #    img_aug = cv2.add(img_aug, noise)

        # Save with incremented name
        filename = f"{base_name}_{start_index + i:04d}.jpg"
        cv2.imwrite(os.path.join(output_folder, filename), img_aug)

    print(f"[INFO] Saved {augment_count} images starting from index {start_index} in {output_folder}")


In [12]:
#"C:\Users\josep\OneDrive\Desktop\noWave\noWave1.jpg"
# === Loop through angles 1 to 7 ===
for i in range(1, 21):  # from 1 to 10 inclusive
    img_path = f"C:/Users/josep/OneDrive/Desktop/noWave/noWave{i}A.jpg"
    img = cv2.imread(img_path)

    if img is None:
        print(f"⚠️ Could not read image: {img_path}")
        continue

    output_folder = "dataset1/noWave"
    base_name = "noWave"

    augment_noWave_image(
        image=img,
        output_folder=output_folder,
        base_name=base_name,
        augment_count=5
    )

    print(f"✅ Augmented angle_{i} and saved to {output_folder}")


[INFO] Saved 5 images starting from index 200 in dataset1/noWave
✅ Augmented angle_1 and saved to dataset1/noWave
[INFO] Saved 5 images starting from index 205 in dataset1/noWave
✅ Augmented angle_2 and saved to dataset1/noWave
[INFO] Saved 5 images starting from index 210 in dataset1/noWave
✅ Augmented angle_3 and saved to dataset1/noWave
[INFO] Saved 5 images starting from index 215 in dataset1/noWave
✅ Augmented angle_4 and saved to dataset1/noWave
[INFO] Saved 5 images starting from index 220 in dataset1/noWave
✅ Augmented angle_5 and saved to dataset1/noWave
[INFO] Saved 5 images starting from index 225 in dataset1/noWave
✅ Augmented angle_6 and saved to dataset1/noWave
[INFO] Saved 5 images starting from index 230 in dataset1/noWave
✅ Augmented angle_7 and saved to dataset1/noWave
[INFO] Saved 5 images starting from index 235 in dataset1/noWave
✅ Augmented angle_8 and saved to dataset1/noWave
[INFO] Saved 5 images starting from index 240 in dataset1/noWave
✅ Augmented angle_9 and

In [13]:
import os
import shutil
import random

# Paths
source_root = "dataset1"
target_root = "theDataset1"
splits = {"Train": 0.70, "Validation": 0.15, "Test": 0.15}

# Set seed for reproducibility
random.seed(42)

# Make destination folders
for split in splits:
    for class_name in ["angle_1", "angle_2", "angle_3", "angle_4", "angle_5", "angle_6", "noWave"]:
        os.makedirs(os.path.join(target_root, split, class_name), exist_ok=True)

# Process each class
for class_name in ["angle_1", "angle_2", "angle_3", "angle_4", "angle_5", "angle_6", "noWave"]:
    src_class_path = os.path.join(source_root, class_name)
    all_files = [f for f in os.listdir(src_class_path) if f.lower().endswith((".jpg", ".png"))]
    random.shuffle(all_files)

    total = len(all_files)
    train_end = int(total * splits["Train"])
    val_end = train_end + int(total * splits["Validation"])

    split_ranges = {
        "Train": all_files[:train_end],
        "Validation": all_files[train_end:val_end],
        "Test": all_files[val_end:]
    }

    for split, files in split_ranges.items():
        for fname in files:
            src = os.path.join(src_class_path, fname)
            dst = os.path.join(target_root, split, class_name, fname)
            shutil.copy2(src, dst)

print("✅ Dataset split complete.")


✅ Dataset split complete.
